In [186]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    Doc
)
from rdflib import Graph, Namespace, URIRef

In [187]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

In [188]:
text_IT = open(r'C:\Users\Юлия\OneDrive\Документы\Универ\3 курс\8 триместр\Курсовая\Semantic annotation\files\text_IT.txt', encoding='UTF-8').read()
text_Friends = open(r'C:\Users\Юлия\OneDrive\Документы\Универ\3 курс\8 триместр\Курсовая\Semantic annotation\files\text_Friends.txt', encoding='UTF-8').read()

# print(f'Текст IT:\n{text_IT}\n\nТекст Fiends:\n{text_Friends}')

In [189]:
ontology_graph = Graph()
ontology_graph.parse(r'C:\Users\Юлия\OneDrive\Документы\Универ\3 курс\8 триместр\Курсовая\Semantic annotation\files\my_ontologies.owl')
# ontology_graph.print()

<Graph identifier=Nf00264d0be5743a48747e4744c389dfe (<class 'rdflib.graph.Graph'>)>

In [190]:
doc_IT = Doc(text_IT)
doc_Friends = Doc(text_Friends)

In [191]:
doc_IT.segment(segmenter)
doc_Friends.segment(segmenter)

In [192]:
doc_IT.tag_morph(morph_tagger)
doc_Friends.tag_morph(morph_tagger)

In [193]:
doc_IT.parse_syntax(syntax_parser)
doc_Friends.parse_syntax(syntax_parser)

In [194]:
for token in doc_IT.tokens:
    token.lemmatize(morph_vocab)

for token in doc_Friends.tokens:
    token.lemmatize(morph_vocab)

lemmas_doc_IT = []
lemmas_doc_Friends = []

for token in doc_IT.tokens:
    if token.pos not in {'PUNCT', 'PRED', 'CONJ', 'PART', 'CCONJ', 'SCONJ', 'ADP'}:
        lemmas_doc_IT.append(token.lemma.upper())

for token in doc_Friends.tokens:
    if token.pos not in {'PUNCT', 'PRED', 'CONJ', 'PART', 'CCONJ', 'SCONJ', 'ADP'}:
        lemmas_doc_Friends.append(token.lemma.upper())

print(lemmas_doc_IT)
print(lemmas_doc_Friends)

['КОМПАНИЯ', 'РЕШАТЬ', 'МНОЖЕСТВО', 'ПРОБЛЕМА', 'ИТ-СФЕРА', 'ОНА', 'ОБЪЕДИНЯТЬ', 'ЭКСПЕРТ', 'СОЗДАНИЕ', 'ИННОВАЦИОННЫЙ', 'РЕШЕНИЕ', 'ОНА', 'ВАЖНЫЙ', 'СИНХРОНИЗИРОВАТЬ', 'УСИЛИЕ', 'КОМАНДА', 'ДОСТИГНУТЬ', 'ПОСТАВИТЬ', 'ЦЕЛЬ', 'КАЖДЫЙ', 'СОТРУДНИК', 'КЛЮЧЕВОЙ', 'ЗВЕНО', 'ВНОСИТЬ', 'СВОЙ', 'ВКЛАД', 'УСПЕХ', 'РАБОТА', 'КОМПАНИЯ', 'ТРЕБОВАТЬ', 'ТЕХНИЧЕСКИЙ', 'НАВЫК', 'УМЕНИЕ', 'РАБОТАТЬ', 'КОЛЛЕКТИВ', 'ВМЕСТЕ', 'МЫ', 'СОЗДАВАТЬ', 'ПРОДУКТ', 'МЕНЯТЬ', 'МИР', 'КОМПАНИЯ', 'РАБОЧИЙ', 'МЕСТО', 'СООБЩЕСТВО', 'ЕДИНОМЫШЛЕННИК', 'СТРЕМИТЬСЯ', 'СОВЕРШЕНСТВО', 'ИТ']
['КОМПАНИЯ', 'ДРУГ', 'КАЖДЫЙ', 'ВСТРЕЧА', 'ПРАЗДНИК', 'МЫ', 'ДЕЛИТЬСЯ', 'РАДОСТЬ', 'СМЕХ', 'ИСТОРИЯ', 'ВМЕСТЕ', 'МЫ', 'ПРОВОДИТЬ', 'ВРЕМЯ', 'ПРИРОДА', 'ГОТОВИТЬ', 'ВКУСНЫЙ', 'БЛЮДО', 'УСТРАИВАТЬ', 'ПИКНИК', 'ПРОДУКТ', 'КОТОРЫЙ', 'МЫ', 'ВЫБИРАТЬ', 'ЛЮБОВЬ', 'СТАНОВИТЬСЯ', 'ЧАСТЬ', 'НАШ', 'ОБЩИЙ', 'СТОЛ', 'ФРУКТ', 'СЫР', 'КАЖДЫЙ', 'ВНОСИТЬ', 'СВОЙ', 'ВКЛАД', 'ТАКОЙ', 'АТМОСФЕРА', 'ДРУЖБА', 'КРЕПНУТЬ', 'ВОСПОМИНАНИЕ', 'СТАНОВИТЬСЯ', 'ЯРЧЕ', '

In [195]:
def get_ontologies(lemmas):
    ontologies_word = {}
    for word in lemmas:
        ontologies_word[word] = []
        synsets = ontology_graph.query(f"""
            SELECT ?synset_id
            WHERE {{
                ?sense rdf:type ns1:sense ;
                    ns1:name ?name ;
                    ns1:lemma ?lemma ;
                    ns1:synset_id ?synset_id .
                FILTER(CONTAINS(?lemma, "{word}"))
            }}
        """)
        for row in synsets:
            synset_id = row['synset_id']

            definitions = ontology_graph.query(f"""
                SELECT ?definition
                WHERE {{
                    ?synset rdf:type ns1:synset ;
                            ns1:id "{synset_id}" ;
                            ns1:definition ?definition .
                }}
            """)

            for d in definitions:
                ontologies_word[word].append(d['definition'])
    return ontologies_word

In [196]:
onologies_doc_IT = get_ontologies(lemmas_doc_IT)
onologies_doc_IT

{'КОМПАНИЯ': [rdflib.term.Literal('группа лиц, проводящих время совместно'),
  rdflib.term.Literal('компания, осуществляющая свою деятельность в области информационных технологий')],
 'РЕШАТЬ': [rdflib.term.Literal('обдумывать ситуацию, формулируя определённый вывод, делая заключение или вырабатывая план какого-то действия')],
 'МНОЖЕСТВО': [],
 'ПРОБЛЕМА': [rdflib.term.Literal('сложный теоретический или практический вопрос, требующий решения, исследования')],
 'ИТ-СФЕРА': [],
 'ОНА': [],
 'ОБЪЕДИНЯТЬ': [],
 'ЭКСПЕРТ': [],
 'СОЗДАНИЕ': [],
 'ИННОВАЦИОННЫЙ': [],
 'РЕШЕНИЕ': [],
 'ВАЖНЫЙ': [],
 'СИНХРОНИЗИРОВАТЬ': [],
 'УСИЛИЕ': [],
 'КОМАНДА': [],
 'ДОСТИГНУТЬ': [],
 'ПОСТАВИТЬ': [],
 'ЦЕЛЬ': [],
 'КАЖДЫЙ': [],
 'СОТРУДНИК': [],
 'КЛЮЧЕВОЙ': [],
 'ЗВЕНО': [],
 'ВНОСИТЬ': [],
 'СВОЙ': [],
 'ВКЛАД': [],
 'УСПЕХ': [],
 'РАБОТА': [],
 'ТРЕБОВАТЬ': [],
 'ТЕХНИЧЕСКИЙ': [],
 'НАВЫК': [],
 'УМЕНИЕ': [],
 'РАБОТАТЬ': [],
 'КОЛЛЕКТИВ': [],
 'ВМЕСТЕ': [],
 'МЫ': [],
 'СОЗДАВАТЬ': [],
 'ПРОДУКТ': [

In [197]:
onologies_doc_Friend = get_ontologies(lemmas_doc_Friends)
onologies_doc_Friend

{'КОМПАНИЯ': [rdflib.term.Literal('группа лиц, проводящих время совместно'),
  rdflib.term.Literal('компания, осуществляющая свою деятельность в области информационных технологий')],
 'ДРУГ': [],
 'КАЖДЫЙ': [],
 'ВСТРЕЧА': [],
 'ПРАЗДНИК': [],
 'МЫ': [],
 'ДЕЛИТЬСЯ': [],
 'РАДОСТЬ': [],
 'СМЕХ': [],
 'ИСТОРИЯ': [],
 'ВМЕСТЕ': [],
 'ПРОВОДИТЬ': [],
 'ВРЕМЯ': [],
 'ПРИРОДА': [],
 'ГОТОВИТЬ': [],
 'ВКУСНЫЙ': [],
 'БЛЮДО': [],
 'УСТРАИВАТЬ': [],
 'ПИКНИК': [],
 'ПРОДУКТ': [rdflib.term.Literal('продовольственные продукты, продовольствие'),
  rdflib.term.Literal('компьютерная программа, предназначенная для выполнения определенной задачи, отличной от той, которая связана с работой самого компьютера, обычно предназначенная для использования конечными пользователями')],
 'КОТОРЫЙ': [],
 'ВЫБИРАТЬ': [],
 'ЛЮБОВЬ': [],
 'СТАНОВИТЬСЯ': [],
 'ЧАСТЬ': [],
 'НАШ': [],
 'ОБЩИЙ': [],
 'СТОЛ': [],
 'ФРУКТ': [],
 'СЫР': [],
 'ВНОСИТЬ': [],
 'СВОЙ': [],
 'ВКЛАД': [],
 'ТАКОЙ': [],
 'АТМОСФЕРА': [],
 'ДРУЖ